In [1]:
from model_selection import *
from preprocessing import *

In [11]:
fp_dev = 'development.csv'
fp_eval = "evaluation.csv"
X_train = load(filepath=fp_dev).pipe(extract_features).pipe(drop_duplicates, drop_long_text=True).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)
X_test = load(filepath=fp_eval).pipe(extract_features).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)

X_train, X_test = text_mining_tfdf(X_train, X_test, min_df=0.1) #if you don't want to use it, just comment it out

X_train, X_test = add_word_embeddings(X_train, X_test) #if you don't want to use it, just comment it out

X_train = X_train.pipe(convert_categorical) #necessary
X_test = X_test.pipe(convert_categorical)   #necessary

print('Start Normalizing')
X_train, X_test, y_train = normalize(X_train, X_test)
print('Finish Normalizing')

100%|██████████| 74999/74999 [00:10<00:00, 7449.97it/s]
/home/christian/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/christian/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn(
/home/christian/.local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Sto facendo autovalidation


Progress: 100.0% Trials:    9 Best score:  1.000000 ETA:   0h 0m 0s
Training again with best arguments
Read 2M words
Number of words:  250614
Number of labels: 2
Progress: 100.0% words/sec/thread: 1074295 lr:  0.000000 avg.loss:  0.014311 ETA:   0h 0m 0s 18.5% words/sec/thread: 1094085 lr:  0.069274 avg.loss:  0.067859 ETA:   0h 0m14s


Start Normalizing
Finish Normalizing


In [12]:
clf = RandomForestClassifier(n_estimators = 2000, criterion = 'entropy', random_state=42, n_jobs=-1)
print('Start Training')
clf.fit(X_train,y_train)
print('Finish Training')

y_pred = clf.predict(X_test)

filepath = "5_text_mining_min_df_01_RF.csv"
save_results(y_pred, fp=filepath)
print("File Salvato!")

Start Training
Finish Training
File Salvato!


In [15]:
importances = pd.Series(X_train.columns, clf.feature_importances_)
importances.sort_index(ascending = False).head(30)

0.385126    embedding_negativity
0.373139    embedding_positivity
0.071149                     ids
0.027799                compound
0.022211                     neg
0.016811                     pos
0.014785                polarity
0.013224              char_count
0.012120            day_of_month
0.009588                     neu
0.009331             hour_of_day
0.007192            subjectivity
0.006203                       @
0.005697           month_of_year
0.004535             day_of_week
0.004158                       .
0.003954                       !
0.003475                     n't
0.003198                       ,
0.002379                       ?
0.002107                     ...
0.001820                      's
dtype: object

In [18]:
for col in X_train.columns:
    print(col) 

ids
day_of_week
month_of_year
day_of_month
hour_of_day
char_count
neg
neu
pos
compound
polarity
subjectivity
!
's
,
.
...
?
@
n't
embedding_negativity
embedding_positivity


In [9]:
clf = LinearSVC()
print('Start Training')
clf.fit(X_train,y_train)
print('Finish Training')

y_pred = clf.predict(X_test)

filepath = "4_notextmining_LSVC.csv"
save_results(y_pred, fp=filepath)
print("File Salvato!")

Start Training
Finish Training
File Salvato!


In [33]:
X_train_1

,ids,day_of_week,month_of_year,day_of_month,hour_of_day,char_count,neg,neu,pos,compound,...,!,'s,",",.,...,?,@,n't,embedding_negativity,embedding_positivity
0,0.425080,0.000000,0.5,0.566667,0.043478,0.930070,0.092,0.625,0.283,0.781884,...,0.000000,0.333333,0.634648,0.000000,0.32249,0.00000,0.623707,0.000000,1.552532e-06,9.999985e-01
1,0.594974,1.000000,0.5,1.000000,0.260870,0.573427,0.000,0.768,0.232,0.805978,...,0.000000,0.000000,0.000000,0.899931,0.00000,0.00000,0.434056,0.000000,3.202391e-04,9.996798e-01
2,0.611332,0.000000,1.0,0.000000,0.478261,0.237762,0.000,1.000,0.000,0.499110,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,1.000000,0.000000,8.297120e-06,9.999917e-01
3,0.414373,1.000000,0.5,0.533333,0.086957,0.321678,0.000,0.722,0.278,0.703401,...,0.000000,0.000000,0.000000,0.410761,0.00000,0.00000,0.812129,0.430331,9.999447e-01,5.536529e-05
4,0.619215,0.166667,1.0,0.033333,0.000000,0.433566,0.000,0.843,0.157,0.538352,...,0.000000,0.000000,0.000000,0.410761,0.00000,0.40837,0.812129,0.000000,9.999999e-01,6.130813e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222895,0.921197,0.833333,1.0,0.633333,0.869565,0.496503,0.000,0.563,0.437,0.822854,...,0.000000,0.000000,0.000000,0.000000,0.45607,0.00000,0.891914,0.000000,9.128424e-01,8.715758e-02
222896,0.605527,0.000000,1.0,0.000000,0.043478,0.132867,0.000,1.000,0.000,0.499110,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,1.000000,0.000000,9.249561e-10,1.000000e+00
222897,0.607632,0.000000,1.0,0.000000,0.260870,0.468531,0.227,0.625,0.148,0.326285,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,1.000000,0.000000,9.999924e-01,7.683189e-06
222898,0.896096,0.666667,1.0,0.600000,0.347826,0.321678,0.612,0.388,0.000,0.063336,...,0.707298,0.000000,0.000000,0.000000,0.00000,0.00000,0.700135,0.000000,9.999993e-01,6.642404e-07


In [35]:
tweets = load('development.csv')
y = tweets.pop('sentiment')
X = tweets

X_train_1, X_test_1, y_train_1, y_test = train_test_split(X, y, train_size= 0.8, test_size=0.2, random_state= 42)
X_train_1 = pd.concat([X_train_1, y_train_1], axis=1)

X_train = X_train_1.pipe(extract_features).pipe(drop_duplicates, drop_long_text=True).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)
X_test = X_test_1.pipe(extract_features).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)

#X_train, X_test = text_mining_tfdf(X_train, X_test) #if you don't want to use it, just comment it out

X_train, X_test = add_word_embeddings(X_train, X_test) #if you don't want to use it, just comment it out

X_train = X_train.pipe(convert_categorical) #necessary
X_test = X_test.pipe(convert_categorical)   #necessary

print('Start Normalizing')
X_train, X_test, y_train = normalize(X_train, X_test)
print('Finish Normalizing')


/home/christian/Desktop/DLS-Project/DSL2122_january_dataset/preprocessing.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['text'] = tweets['text'].str.replace(pat= pat, repl = repl, regex=True)
100%|██████████| 178472/178472 [11:14<00:00, 264.42it/s]
/home/christian/Desktop/DLS-Project/DSL2122_january_dataset/preprocessing.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets["neg"] = sentiment[:, 0]
/home/christian/Desktop/DLS-Project/DSL2122_january_dataset/preprocessing.py:185: Settin

Sto facendo autovalidation


Progress: 100.0% Trials:   12 Best score:  0.999993 ETA:   0h 0m 0s
Training again with best arguments
Read 2M words
Number of words:  215890
Number of labels: 2
Progress: 100.0% words/sec/thread: 1242399 lr:  0.000000 avg.loss:  0.013965 ETA:   0h 0m 0s


Start Normalizing
Finish Normalizing


In [39]:
# Use the random grid to search for best hyperparameters
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier()

parameters_random_forest = {
    "n_estimators" : np.arange(start= 300, stop = 1500, step = 300), 
    "max_features" : ["sqrt", "log2", None, 0.3, 0.4, 0.5, 0.6], 
    #"max_depth ": np.arange(start= 15, stop = 30, step = 5), 
    "criterion": ["gini", "entropy"],
    "min_samples_split" :np.arange(start= 2, stop = 20, step = 3),
    "min_samples_leaf":np.arange(start= 2, stop = 20, step = 3),
    "bootstrap" : [True, False]
}

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = parameters_random_forest, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring= 'f1_macro')
# Fit the random search model
rf_random.fit(X_train, y_train)

print(rf_random.best_params_)
print()
print(rf_random.best_score_)
print()
print(rf_random.best_estimator_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=True, criterion=entropy, max_features=0.6, min_samples_leaf=14, min_samples_split=17, n_estimators=300; total time= 3.9min
[CV] END bootstrap=True, criterion=entropy, max_features=0.6, min_samples_leaf=14, min_samples_split=17, n_estimators=300; total time= 4.1min
[CV] END bootstrap=True, criterion=entropy, max_features=0.6, min_samples_leaf=14, min_samples_split=17, n_estimators=300; total time= 4.1min
[CV] END bootstrap=True, criterion=entropy, max_features=sqrt, min_samples_leaf=17, min_samples_split=8, n_estimators=900; total time= 5.2min
[CV] END bootstrap=True, criterion=entropy, max_features=sqrt, min_samples_leaf=17, min_samples_split=8, n_estimators=900; total time= 5.3min
[CV] END bootstrap=True, criterion=entropy, max_features=sqrt, min_samples_leaf=17, min_samples_split=8, n_estimators=900; total time= 5.3min
[CV] END bootstrap=True, criterion=gini, max_features=0.4, min_samples_leaf=11, min_s

/home/christian/.local/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, criterion=entropy, max_features=0.5, min_samples_leaf=8, min_samples_split=8, n_estimators=900; total time=11.5min
[CV] END bootstrap=False, criterion=gini, max_features=0.6, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=29.1min
[CV] END bootstrap=True, criterion=entropy, max_features=0.5, min_samples_leaf=8, min_samples_split=8, n_estimators=900; total time=11.6min
[CV] END bootstrap=False, criterion=entropy, max_features=0.4, min_samples_leaf=11, min_samples_split=17, n_estimators=600; total time= 8.3min
[CV] END bootstrap=False, criterion=entropy, max_features=0.4, min_samples_leaf=11, min_samples_split=17, n_estimators=600; total time= 8.7min
[CV] END bootstrap=False, criterion=entropy, max_features=log2, min_samples_leaf=11, min_samples_split=8, n_estimators=900; total time= 7.9min
[CV] END bootstrap=False, criterion=entropy, max_features=log2, min_samples_leaf=11, min_samples_split=8, n_estimators=900; total time= 8.0min
[CV] END 

In [40]:
print(rf_random.best_params_)
print()
print(rf_random.best_score_)
print()
print(rf_random.best_estimator_)

{'n_estimators': 1200, 'min_samples_split': 14, 'min_samples_leaf': 14, 'max_features': 'sqrt', 'criterion': 'entropy', 'bootstrap': True}

0.9674699361940174

RandomForestClassifier(criterion='entropy', max_features='sqrt',
                       min_samples_leaf=14, min_samples_split=14,
                       n_estimators=1200)
